In [ ]:
# 导入常用计算库
import numpy as np
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10

In [ ]:
# 设置超参数
batch_size = 32    # 小批量梯度下降的批规模
nb_classes = 10    # 数据集有10个类别的图片
nb_epoch = 50      # 进行50轮迭代优化
nb_filters = [32, 32, 64, 64]    # 四个卷积层里卷积核的个数
pool_size = (2,2)                # 池化操作的窗口大小
kernel_size = (3,3)              # 卷积核尺寸

In [ ]:
# 读取训练/测试数据
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
# 数据归一化
X_train = X_train.astype('float16') / 255
X_test = X_test.astype('float16') / 255

In [ ]:
# 标签独热编码
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
# 创建数据生成器
datagen = ImageDataGenerator(featurewise_center=False, samplewise_center=False, 
                             featurewise_std_normalization=False, samplewise_std_normalization=False,
                             zca_whitening=False, rotation_range=0, width_shift_range=0.1, height_shift_range=0.1,
                             horizontal_flip=True, vertical_flip=False)
datagen.fit(X_train)

In [ ]:
# 搭建神经网络模型
model = Sequential()

# 输入（卷积）层
model.add(Conv2D(nb_filters[0], kernel_size, padding='same', input_shape=X_train.shape[1:], activation='relu'))

# 卷积+池化+Dropout
model.add(Conv2D(nb_filters[1], kernel_size, activation='relu'))
model.add(MaxPool2D(pool_size=pool_size))
model.add(Dropout(0.25))

# 卷积+池化+Dropout
model.add(Conv2D(nb_filters[2], kernel_size, padding='same',  activation='relu'))
model.add(Conv2D(nb_filters[3], kernel_size, activation='relu'))
model.add(MaxPool2D(pool_size=pool_size))
model.add(Dropout(0.25))

In [ ]:
# 卷积层展开为全连接层
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

# softmax分类
model.add(Dense(nb_classes, activation='softmax'))

In [ ]:
# Adam梯度下降法
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
#best_model = ModelCheckpoint('cifar10_base.h5', monitor='val_loss', verbose=0, save_best_only=True)
#tb = TensorBoard(log_dir='./logs')

In [ ]:
# 启动训练，用测试集作验证集
model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    epochs=nb_epoch, verbose=1,
                    validation_data=(X_test, y_test))